In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2019-12-31_14:56
E:\RIS\Staff Folders\Samuel\Requests\SH\Speed_Data_Viewer


In [2]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')



Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-31 14:56:40


In [14]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-31 15:02:15


### 1. Sip 702
#### Created table in Pgsql containing all nysdot_all data for Sip: 702 - ST NICHOLAS AVE BROADWAY TO W 180TH ST

In [15]:
cdb.dfquery(""" 

select * from sam_crashes""")

,gid,case_num,case_yr,ref_mrkr,accd_dte,road_sys,num_of_fat,num_of_inj,reportable,police_dep,...,boro,st_nm,alis_node,alis_mastr,x,y,st_x,st_y,wrong_pct,cleanname
0,32797.0,36074034,2015,None,2015-11-04,12,0,0,Y,3030,...,1,SAINT NICHOLAS AVENUE,9999,9999,1002368.18954493,248319.604179096,1002380.80637108,248347.276013844,None,SAINT NICHOLAS AVENUE
1,33640.0,35712624,2015,None,2015-01-23,12,0,0,Y,3030,...,1,BROADWAY,9999,9999,1001058.81418762,246203.903533388,1001120.52677892,246167.475482211,None,BROADWAY
2,34149.0,35779512,2015,None,2015-04-29,12,0,6,Y,3030,...,1,None,9999,9999,1002189.98757374,247994.361110338,1002185.97768511,247999.427118242,None,None
3,34814.0,35893252,2015,None,2015-08-29,12,0,1,Y,3030,...,1,None,9999,9999,1002080.94107652,247761.225576307,1002061.57144333,247769.319648951,None,None
4,34919.0,36233898,2016,None,2016-05-26,12,0,1,Y,3030,...,1,None,9999,9999,1001286.81944414,246376.729277162,1001288.48247236,246375.158784747,None,None
5,35369.0,36322851,2016,None,2016-07-24,12,0,1,Y,3030,...,1,None,9999,9999,1001058.81418623,246203.903153768,1001120.52677892,246167.475482211,None,None
6,35370.0,36322854,2016,None,2016-07-25,12,0,1,Y,3030,...,1,None,9999,9999,1001676.04137095,247058.715456856,1001668.86260037,247062.893599465,None,None
7,35555.0,36055273,2015,None,2015-12-05,12,0,1,Y,0,...,1,SAINT NICHOLAS AVENUE,9999,9999,1002063.44205809,247770.921941237,1002061.57144333,247769.319648951,None,SAINT NICHOLAS AVENUE
8,35707.0,36410717,2016,None,2016-09-30,12,0,3,Y,3030,...,1,None,9999,9999,1002189.98757182,247994.360728686,1002185.97768511,247999.427118242,None,None
9,35881.0,36457565,2016,None,2016-11-04,12,0,4,Y,3030,...,1,BROADWAY,9999,9999,1001015.55350842,245944.822005475,1001035.61553515,245930.077673495,None,BROADWAY


### Injury Summary

In [71]:
inj_sum = cdb.dfquery("""

select *, ("Severe Injuries"  + "Fatalities") as KSI from (

select * from (
--Injuries/Severity By Mode
select  case when accd_type_int = 1 then 'ped'
         when accd_type_int = 2 then 'bicycle'
         when accd_type_int = 3 then 'mvo' end as "Type"
    ,sum(num_of_inj) as "Total Injuries"
    ,sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as "Severe Injuries"
    ,0 as "Fatalities"
from sam_crashes
group by  accd_type_int
order by  accd_type_int
) x

union 

select * from (
-- All fatalities on stretch
select case when pos = 'PD' then 'ped'
        when pos = 'BI' then 'bicycle'
        when pos in ('MO','PS','DR') then 'mvo' end as "Type"
        ,0 as "Total Injuries"
        ,0 as "Severe Injuries"
        ,count(id_) as "Fatalities" 
from public.fatality_nycdot_current fat
join archive."18d.2019-11-13_lion" lion
on fat.segmentid = lion.segmentid
where lion.mft in (
        select mft
        from archive."18d.2019-11-13_lion" lion
        where lion.segmentid::int in (70923,70925,70927,70951,70953,70957,71088,71121,111044,111222,111226,111227,111228,173841,173842,173843,173862,173863,173864)
        )
group by pos 

union 



select case when pos = 'PD' then 'ped'
        when pos = 'BI' then 'bicycle'
        when pos in ('MO','PS','DR') then 'mvo' end as "Type"
        ,0 as "Total Injuries"
        ,0 as "Severe Injuries"
        ,count(id_) as "Fatalities"
from public.fatality_nycdot_current fat
where masterid in (
        select masteridfrom mid
        from archive."18d.2019-11-13_lion" lion
        where mft in (
        select mft
        from archive."18d.2019-11-13_lion" lion
        where lion.segmentid::int in (70923,70925,70927,70951,70953,70957,71088,71121,111044,111222,111226,111227,111228,173841,173842,173843,173862,173863,173864)
        )


        union 

        select masteridto mid
        from archive."18d.2019-11-13_lion" lion
        where mft in (
        select mft
        from archive."18d.2019-11-13_lion" lion
        where lion.segmentid::int in (70923,70925,70927,70951,70953,70957,71088,71121,111044,111222,111226,111227,111228,173841,173842,173843,173862,173863,173864)
        ) 
        )
group by pos

) y



) total 
""")

In [72]:
inj_sum

,Type,Total Injuries,Severe Injuries,Fatalities,ksi
0,bicycle,15,1,0,1
1,ped,40,5,0,5
2,mvo,90,4,0,4


In [73]:
tot = inj_sum.sum(numeric_only=True)
tot['Type']='Total'

In [74]:
inj_sum.append(tot, ignore_index = True)

,Type,Total Injuries,Severe Injuries,Fatalities,ksi
0,bicycle,15,1,0,1
1,ped,40,5,0,5
2,mvo,90,4,0,4
3,Total,145,10,0,10


### Fatalities

In [77]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-02 16:02:32


In [84]:
fat = cdb.dfquery("""



select case when pos = 'PD' then 'ped'
        when pos = 'BI' then 'bicycle'
        when pos in ('MO','PS','DR') then 'mvo' end as "Type"
      ,count(case when date_part('year',acdate) between 2013 and 2017 then id_ end) as "2013-2017 (5 Years)"
      ,count(case when date_part('year',acdate)>=2013 and acdate < '07-07-2019'::date then id_ end) as "2013-07/07/2019 (7 Years)"
from public.fatality_nycdot_current fat
join archive."18d.2019-11-13_lion" lion
on fat.segmentid = lion.segmentid
where lion.mft in (
        select mft
        from archive."18d.2019-11-13_lion" lion
        where lion.segmentid::int in (70923,70925,70927,70951,70953,70957,71088,71121,111044,111222,111226,111227,111228,173841,173842,173843,173862,173863,173864)
        )
group by pos

union 



select case when pos = 'PD' then 'ped'
        when pos = 'BI' then 'bicycle'
        when pos in ('MO','PS','DR') then 'mvo' end as "Type"
      ,count(case when date_part('year',acdate) between 2013 and 2017 then id_ end) as "2013-2017 (5 Years)"
      ,count(case when date_part('year',acdate)>=2013 and acdate < '07-07-2019'::date then id_ end) as "2013-07/07/2019 (7 Years)"
from public.fatality_nycdot_current fat
where masterid in (
        select masteridfrom mid
        from archive."18d.2019-11-13_lion" lion
        where mft in (
        select mft
        from archive."18d.2019-11-13_lion" lion
        where lion.segmentid::int in (70923,70925,70927,70951,70953,70957,71088,71121,111044,111222,111226,111227,111228,173841,173842,173843,173862,173863,173864)
        )


        union 

        select masteridto mid
        from archive."18d.2019-11-13_lion" lion
        where mft in (
        select mft
        from archive."18d.2019-11-13_lion" lion
        where lion.segmentid::int in (70923,70925,70927,70951,70953,70957,71088,71121,111044,111222,111226,111227,111228,173841,173842,173843,173862,173863,173864)
        ) 
        )
group by pos






""") 


In [85]:
fat

,Type,2013-2017 (5 Years),2013-07/07/2019 (7 Years)


### Non-Fatal Injuries By Severity 

In [86]:
non_fat_sev = cdb.dfquery("""



select * from (
select 
'A' as Severity,
sum(case when accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Pedestrian,
sum(case when accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Bicyclist,
sum(case when accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) "Motor Vehicle",
sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as Total
FROM sam_crashes

union

select 
'B' as Severity,
sum(case when accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Pedestrian,
sum(case when accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Bicyclist,
sum(case when accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) "Motor Vehicle",
sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text))) as Total
FROM sam_crashes

union

select 
'C' as Severity,
sum(case when accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Pedestrian,
sum(case when accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Bicyclist,
sum(case when accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) "Motor Vehicle",
sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text))) as Total
FROM sam_crashes

union

select 
'Unknown' as Severity,
sum(case when accd_type_int = 1 and coalesce(length(ext_of_inj::text),0) != num_of_inj then num_of_inj - coalesce(length(ext_of_inj::text),0) end) Pedestrian,
sum(case when accd_type_int = 2 and coalesce(length(ext_of_inj::text),0) != num_of_inj then num_of_inj - coalesce(length(ext_of_inj::text),0)  end) Bicyclist,
sum(case when accd_type_int = 3 and coalesce(length(ext_of_inj::text),0) != num_of_inj then num_of_inj - coalesce(length(ext_of_inj::text),0)  end) "Motor Vehicle",
sum(num_of_inj -coalesce(length(ext_of_inj::text),0)) as Total
FROM sam_crashes

)sev
order by severity


""")

In [88]:
non_fat_sev.fillna(0)

,severity,pedestrian,bicyclist,Motor Vehicle,total
0,A,5.0,1.0,4,10
1,B,9.0,4.0,3,16
2,C,26.0,10.0,78,114
3,Unknown,0.0,0.0,5,5


### Injuries by Year

In [92]:
inj_year = cdb.dfquery("""

select * from (
with data as (
select 
case_yr::text as Year,
coalesce(sum(case when accd_type_int = 1 then num_of_inj end),0) "Pedestrian", 
coalesce(sum(case when accd_type_int = 2 then num_of_inj end),0) "Bicyclist",
coalesce(sum(case when accd_type_int = 3 then num_of_inj end),0) "Motor Vehicle"
from sam_crashes
group by case_yr
order by case_yr
)

select data.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
from data

union 

select tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
from (select 'Total' as Year, 
      sum(data."Pedestrian") "Pedestrian",
      sum(data."Bicyclist") "Bicyclist",
      sum(data."Motor Vehicle") "Motor Vehicle"
      from data
    ) tot
)inj_year
order by year

""")


In [93]:
inj_year

,year,Pedestrian,Bicyclist,Motor Vehicle,total
0,2013,15,3,21,39
1,2014,9,6,16,31
2,2015,7,0,14,21
3,2016,5,2,22,29
4,2017,4,4,17,25
5,Total,40,15,90,145


### Injuries By Time of Day By Type

In [96]:
inj_tod = cdb.dfquery("""


select * from (
with data as (
select 
case when date_part('hour',accd_tme) between 0 and 2 then '00:00-03:00' 
     when date_part('hour',accd_tme) between 3 and 5 then '03:00-06:00' 
     when date_part('hour',accd_tme) between 6 and 8 then '06:00-09:00' 
     when date_part('hour',accd_tme) between 9 and 11 then '09:00-12:00' 
     when date_part('hour',accd_tme) between 12 and 14 then '12:00-15:00' 
     when date_part('hour',accd_tme) between 15 and 17 then '15:00-18:00' 
     when date_part('hour',accd_tme) between 18 and 20 then '18:00-21:00' 
     when date_part('hour',accd_tme) between 21 and 23 then '21:00-24:00'
     when date_part('hour',accd_tme) is null then 'Unknown' end as Time,
coalesce(sum(case when accd_type_int = 1 then num_of_inj end),0) "Pedestrian", 
coalesce(sum(case when accd_type_int = 2 then num_of_inj end),0) "Bicyclist",
coalesce(sum(case when accd_type_int = 3 then num_of_inj end),0) "Motor Vehicle"
from sam_crashes
group by case when date_part('hour',accd_tme) between 0 and 2 then '00:00-03:00' 
     when date_part('hour',accd_tme) between 3 and 5 then '03:00-06:00' 
     when date_part('hour',accd_tme) between 6 and 8 then '06:00-09:00' 
     when date_part('hour',accd_tme) between 9 and 11 then '09:00-12:00' 
     when date_part('hour',accd_tme) between 12 and 14 then '12:00-15:00' 
     when date_part('hour',accd_tme) between 15 and 17 then '15:00-18:00' 
     when date_part('hour',accd_tme) between 18 and 20 then '18:00-21:00' 
     when date_part('hour',accd_tme) between 21 and 23 then '21:00-24:00'
     when date_part('hour',accd_tme) is null then 'Unknown'  end
 order by case when date_part('hour',accd_tme) between 0 and 2 then '00:00-03:00' 
     when date_part('hour',accd_tme) between 3 and 5 then '03:00-06:00' 
     when date_part('hour',accd_tme) between 6 and 8 then '06:00-09:00' 
     when date_part('hour',accd_tme) between 9 and 11 then '09:00-12:00' 
     when date_part('hour',accd_tme) between 12 and 14 then '12:00-15:00' 
     when date_part('hour',accd_tme) between 15 and 17 then '15:00-18:00' 
     when date_part('hour',accd_tme) between 18 and 20 then '18:00-21:00' 
     when date_part('hour',accd_tme) between 21 and 23 then '21:00-24:00'
     when date_part('hour',accd_tme) is null then 'Unknown'  end)


select data.*, 
"Pedestrian"+"Bicyclist"+"Motor Vehicle" Total,
(round((("Pedestrian"+"Bicyclist"+"Motor Vehicle")::float/(select sum(num_of_inj) from sam_crashes)::float)::numeric, 3)*100.0)::text as "Percent of Known Crashes"
from data

union 

select tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total, 'N/A' as "Percent of Known Crashes"
from (select 'Total' as Time, 
      sum(data."Pedestrian") "Pedestrian",
      sum(data."Bicyclist") "Bicyclist",
      sum(data."Motor Vehicle") "Motor Vehicle"
      from data
      ) tot
)inj_tod
order by time


""")

In [97]:
inj_tod

,time,Pedestrian,Bicyclist,Motor Vehicle,total,Percent of Known Crashes
0,00:00-03:00,5,1,18,24,16.6000
1,03:00-06:00,1,0,3,4,2.8000
2,06:00-09:00,2,0,4,6,4.1000
3,09:00-12:00,4,2,6,12,8.3000
4,12:00-15:00,3,2,19,24,16.6000
5,15:00-18:00,6,5,18,29,20.0000
6,18:00-21:00,16,0,12,28,19.3000
7,21:00-24:00,3,5,10,18,12.4000
8,Total,40,15,90,145,N/A


### Pedestrian Injuries By Control And Pedestrian Action

In [100]:
ped_inj_con = cdb.dfquery("""


select * from (
with data as (
select 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing With Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN 'Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN 'Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN 'Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Other Control Types'
     ELSE 'Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
from sam_crashes
where accd_type_int = 1
group by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing With Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN 'Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN 'Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN 'Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Other Control Types'
     ELSE 'Unknown/Indeterminate' END

)

select data.*, 
(round((("Number of Injuries")/(select sum("Number of Injuries") from data where data." " != 'Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
from data
where data." " != 'Unknown/Indeterminate'

union

select data.*, 'N/A' as "Percent of Known Injuries"
from data
where data." " = 'Unknown/Indeterminate'

union

select 
'Total Known' as " " 
, sum("Number of Injuries") as "Number of Injuries"
, (round((sum("Number of Injuries")/(select sum("Number of Injuries") from data where data." " != 'Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
from data
where data." " != 'Unknown/Indeterminate'

union


select 
'Total' as " " 
, sum("Number of Injuries") as "Number of Injuries"
, 'N/A' as "Percent of Known Injuries"
from data
) ped_inj_con
order by "Number of Injuries" 


""")

In [101]:
ped_inj_con

,,Number of Injuries,Percent of Known Injuries
0,Signalized Intersection: Crossing Against Signal,3,8.6000
1,Other Actions/Uncontrolled Intersection,3,8.6000
2,Unknown/Indeterminate,5,N/A
3,Midblock,7,20.0000
4,Signalized Intersection: Crossing With Signal,22,62.9000
5,Total Known,35,100.0000
6,Total,40,N/A


### Bicycle Injuries By Intersection Control and Bicyclist Action

In [102]:
bi_inj_con = cdb.dfquery("""


select * from (
with data as (
select 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing With Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN 'Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN 'Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN 'Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Other Control Types'
     ELSE 'Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
from sam_crashes
where accd_type_int = 2
group by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing With Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN 'Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN 'Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN 'Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN 'Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN 'Other Control Types'
     ELSE 'Unknown/Indeterminate' END

)

select data.*, 
(round((("Number of Injuries")/(select sum("Number of Injuries") from data where data." " != 'Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
from data
where data." " != 'Unknown/Indeterminate'

union

select data.*, 'N/A' as "Percent of Known Injuries"
from data
where data." " = 'Unknown/Indeterminate'

union

select 
'Total Known' as " " 
, sum("Number of Injuries") as "Number of Injuries"
, (round((sum("Number of Injuries")/(select sum("Number of Injuries") from data where data." " != 'Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
from data
where data." " != 'Unknown/Indeterminate'

union


select 
'Total' as " " 
, sum("Number of Injuries") as "Number of Injuries"
, 'N/A' as "Percent of Known Injuries"
from data
) ped_inj_con
order by "Number of Injuries" 


""")

In [103]:
bi_inj_con

,,Number of Injuries,Percent of Known Injuries
0,Signalized Intersection: Crossing Against Signal,2,33.3000
1,Other Actions/Uncontrolled Intersection,4,66.7000
2,Total Known,6,100.0000
3,Unknown/Indeterminate,9,N/A
4,Total,15,N/A


In [104]:
mvo_inj_coll = cdb.dfquery("""

select * from (
with data as(
select 
CASE WHEN collision_ = '03' THEN '1. Left Turn'
     WHEN collision_ = '05' THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     ELSE 'Unknown' END as " ",
sum(num_of_inj) as "Number of Injuries"
from sam_crashes
where accd_type_int = 3
group by CASE WHEN collision_ = '03' THEN '1. Left Turn'
     WHEN collision_ = '05' THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     ELSE 'Unknown' END
order by " "
)


select data.*, 
(round((("Number of Injuries")/(select sum("Number of Injuries") from data where data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
from data
where data." " != 'Unknown'

union

select data.*, 'N/A' as "Percent of Known Injuries"
from data
where data." " = 'Unknown'

union

select 
'9. Total Known' as " " 
, sum("Number of Injuries") as "Number of Injuries"
, (round((sum("Number of Injuries")/(select sum("Number of Injuries") from data where data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
from data
where data." " != 'Unknown'

union


select 
'Total' as " " 
, sum("Number of Injuries") as "Number of Injuries"
, 'N/A' as "Percent of Known Injuries"
from data
) ped_inj_con
order by " " 


"""
                          )

In [105]:
mvo_inj_coll

,,Number of Injuries,Percent of Known Injuries
0,1. Left Turn,1,1.2000
1,2. Right Turn,1,1.2000
2,3. Rear-End,36,43.9000
3,4. Right-Angle,8,9.8000
4,5. Sideswipe(Same Direction),16,19.5000
5,6. Head-on,3,3.7000
6,8. Other Known,17,20.7000
7,9. Total Known,82,100.0000
8,Total,90,N/A
9,Unknown,8,N/A


### Injuries By Age Group

In [107]:
inj_age = cdb.dfquery("""


select * from (
with data as(
select 
CASE WHEN age between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
     WHEN age between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
     WHEN age between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
     WHEN age between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
     ELSE '5. Unknown' End "Age Group",
coalesce(sum(case when accd_type_int = 1 and veh_typ = '6' then num_of_inj end),0) "Pedestrian", 
coalesce(sum(case when accd_type_int = 2 and veh_typ = '5' then num_of_inj end),0) "Bicyclist"
from sam_crashes
join archive."2019_11_13_nysdot_vehicle" nys_v 
    on sam_crashes.crashid = nys_v.crashid
group by CASE WHEN age between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
     WHEN age between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
     WHEN age between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
     WHEN age between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
     ELSE '5. Unknown' End
order by "Age Group"
)


select data.*, "Pedestrian"+"Bicyclist" Total
from data

union 

select tot.*, "Pedestrian"+"Bicyclist" Total
from (select 'Total' as Year, 
      sum(data."Pedestrian") "Pedestrian",
      sum(data."Bicyclist") "Bicyclist"
      from data
    ) tot

) inj_age
order by "Age Group"

""")

In [108]:
inj_age

,Age Group,Pedestrian,Bicyclist,total
0,1. Children(1-17),1,0,1
1,2. Young Adults(18-29),6,7,13
2,3. Adults(30-64),23,6,29
3,4. Seniors(65-120),7,1,8
4,5. Unknown,3,1,4
5,Total,40,15,55


### Ped Injuries By Veh Action And Pre Action

In [109]:
ped_veh_actn = cdb.dfquery("""


SELECT * FROM (
with data as(
select 
CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
     WHEN pre_accd_actn = '02' THEN '2. Right Turn'
     WHEN pre_accd_actn = '01' THEN '3. Going Straight'
     WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
     WHEN pre_accd_actn = '15' THEN '5. Backing'
     WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'   
     WHEN pre_accd_actn = 'ZZ' THEN '7. Unknown' END as " ",
coalesce(sum(case when ped_actn = '01' and num_of_inj = 1 THEN num_of_inj END),0) as "Crossing With Signal",
coalesce(sum(case when ped_actn = '02' and num_of_inj = 1 THEN num_of_inj END),0) as "Crossing Against Signal",
coalesce(sum(case when ped_actn = '03' and num_of_inj = 1 THEN num_of_inj END),0) as "Crossing No Signal Marked Crosswalk",
coalesce(sum(case when ped_actn = '04' and num_of_inj = 1 THEN num_of_inj END),0) as "Crossing No Signal or Crosswalk",
coalesce(sum(case when ped_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN num_of_inj END),0) as "Others",
coalesce(sum(case when ped_actn in ('??','XX','YY','ZZ') or num_of_inj>1 THEN num_of_inj END),0) as "UnKnown"
from sam_crashes
join archive."2019_11_13_nysdot_vehicle" nys_v 
    on sam_crashes.crashid = nys_v.crashid
where accd_type_int = 1
group by CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
     WHEN pre_accd_actn = '02' THEN '2. Right Turn'
     WHEN pre_accd_actn = '01' THEN '3. Going Straight'
     WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
     WHEN pre_accd_actn = '15' THEN '5. Backing'
     WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'     
     WHEN pre_accd_actn = 'ZZ' THEN '7. Unknown' END
limit  7
)

select data.*, "Crossing With Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
from data

union 

select tot.*, "Crossing With Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
from (select 'Total' as " ", 
      sum(data."Crossing With Signal") "Crossing With Signal",
      sum(data."Crossing Against Signal") "Crossing Against Signal",
      sum(data."Crossing No Signal Marked Crosswalk") "Crossing No Signal Marked Crosswalk",
      sum(data."Crossing No Signal or Crosswalk") "Crossing No Signal or Crosswalk",
      sum(data."Others") "Others",
      sum(data."UnKnown") "UnKnown"
      from data
    ) tot

) ped_veh_actn
order by " "

""")

In [111]:
ped_veh_actn

,,Crossing With Signal,Crossing Against Signal,Crossing No Signal Marked Crosswalk,Crossing No Signal or Crosswalk,Others,UnKnown,total
0,1. Left Turn,14,1,0,0,0,5,20
1,2. Right Turn,0,1,0,0,0,0,1
2,3. Going Straight,5,3,0,1,2,3,14
3,4. Making U Turn,1,0,0,0,0,0,1
4,5. Backing,0,0,0,0,1,0,1
5,6. Other,0,0,0,1,0,0,1
6,7. Unknown,2,0,0,0,0,0,2
7,Total,22,5,0,2,3,8,40
